### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/417_TROCAS7_Fusion_Apr2021_DENOVO_179/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/417_TROCAS7_Fusion_Apr2021_DENOVO_179


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks417 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/417_TROCAS7_Fusion_Apr2021_DENOVO_179/417_TROCAS7_Fusion_Apr2021_DENOVO_179_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks417))

print(peaks417.columns)

# These columns mess things up- get rid of them

del peaks417['Fraction']
del peaks417['Scan']
del peaks417['Source File']
del peaks417['Tag Length']
del peaks417['PTM']
del peaks417['tag (>=0%)']
del peaks417['mode']
del peaks417['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks417.columns = columns

mean_len = peaks417['length'].mean()
print(mean_len)

# look at the dataframe
peaks417.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 259
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
9.749034749034749


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,VLEGNEQFLNAAK,99,13,716.8768,2,75.43,3970000.0,1431.7358,2.3
1,LAADDFR,98,7,404.2042,2,60.45,246000.0,806.3922,1.9
2,KM(+15.99)LNAAK,98,7,396.2247,2,51.67,344000.0,790.4371,-2.8
3,LSSPATLNSR,97,10,523.2874,2,51.73,96600000.0,1044.5564,3.7
4,LSSPATLNSR,97,10,523.2870,2,52.24,96600000.0,1044.5564,2.9


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks417['A'] = peaks417['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks417['C'] = peaks417['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks417['D'] = peaks417['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks417['E'] = peaks417['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks417['F'] = peaks417['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks417['G'] = peaks417['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks417['H'] = peaks417['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks417 output, there will be no isoleucines (they're lumped in with leucines)
peaks417['I'] = peaks417['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks417['K'] = peaks417['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks417['L'] = peaks417['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks417['M'] = peaks417['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks417['N'] = peaks417['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks417['P'] = peaks417['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks417['Q'] = peaks417['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks417['R'] = peaks417['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks417['S'] = peaks417['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks417['T'] = peaks417['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks417['V'] = peaks417['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks417['W'] = peaks417['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks417['Y'] = peaks417['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks417['c-carb'] = peaks417['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks417['m-oxid'] = peaks417['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks417['n-deam'] = peaks417['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks417['q-deam'] = peaks417['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks417['stripped_peptide'] = peaks417['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks417['stripped_length'] = peaks417['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks417['ptm-total'] = peaks417['c-carb'] + peaks417['m-oxid'] + peaks417['n-deam'] + peaks417['q-deam']

# calculate NAAF numerator for each peptide k
peaks417['NAAF_num.'] = peaks417['Area'] / peaks417['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks417.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50.csv")

# check out the results
peaks417.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,VLEGNEQFLNAAK,99,13,716.8768,2,75.43,3970000.0,1431.7358,2.3,2,...,0,0,0,0,0,0,VLEGNEQFLNAAK,13,0,3.053846e+05
1,LAADDFR,98,7,404.2042,2,60.45,246000.0,806.3922,1.9,2,...,0,0,0,0,0,0,LAADDFR,7,0,3.514286e+04
2,KM(+15.99)LNAAK,98,7,396.2247,2,51.67,344000.0,790.4371,-2.8,2,...,0,0,0,1,0,0,KMLNAAK,7,1,4.914286e+04
3,LSSPATLNSR,97,10,523.2874,2,51.73,96600000.0,1044.5564,3.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,9.660000e+06
4,LSSPATLNSR,97,10,523.2870,2,52.24,96600000.0,1044.5564,2.9,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,9.660000e+06


In [5]:
# keep only stripped peptide I/L and NAAF
dn_417 = peaks417[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_417.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_417.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/417B_MN_T24_GD_DN50_ILnaafs.csv")

dn_417.head()

,stripped_peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,3970000.0,3.053846e+05
1,LAADDFR,246000.0,3.514286e+04
2,KMLNAAK,344000.0,4.914286e+04
3,LSSPATLNSR,96600000.0,9.660000e+06
4,LSSPATLNSR,96600000.0,9.660000e+06


In [6]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks417['A'].sum(),
        'C': peaks417['C'].sum(),
        'D': peaks417['D'].sum(),
        'E': peaks417['E'].sum(),
        'F': peaks417['F'].sum(),
        'G': peaks417['G'].sum(),
        'H': peaks417['H'].sum(),
        'I': peaks417['I'].sum(),
        'K': peaks417['K'].sum(),
        'L': peaks417['L'].sum(),
        'M': peaks417['M'].sum(),
        'N': peaks417['N'].sum(),
        'P': peaks417['P'].sum(),
        'Q': peaks417['Q'].sum(),
        'R': peaks417['R'].sum(),
        'S': peaks417['S'].sum(),
        'T': peaks417['T'].sum(),
        'V': peaks417['V'].sum(),
        'W': peaks417['W'].sum(),
        'Y': peaks417['Y'].sum(),
        'c-carb': peaks417['c-carb'].sum(),
        'm-oxid': peaks417['m-oxid'].sum(),
        'n-deam': peaks417['n-deam'].sum(),
        'q-deam': peaks417['q-deam'].sum(),
        'Total area': peaks417['Area'].sum(),
        'Total length': peaks417['stripped_length'].sum()
       }

totalpeaks417 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks417['% C w/ carb'] = totalpeaks417['c-carb'] / totalpeaks417['C'] 

# calculate percentage of M's that are oxidized
totalpeaks417['% M w/ oxid'] = totalpeaks417['m-oxid'] / totalpeaks417['M'] 

# calculate percentage of N's that are deamidated
totalpeaks417['% N w/ deam'] = totalpeaks417['n-deam'] / totalpeaks417['N'] 

# calculate percentage of N's that are deamidated
totalpeaks417['% Q w/ deam'] = totalpeaks417['q-deam'] / totalpeaks417['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks417['NAAF denom.'] = totalpeaks417['Total area'] / totalpeaks417['Total length']

# write modified dataframe to new txt file
totalpeaks417.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN//417B_MN_T24_GD_DN50_totals.csv")

totalpeaks417.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,184,76,87,121,56,146,39,0,207,327,...,67,42,4,2.230072e+09,2276,1.0,0.57265,0.206897,0.066667,979820.765013


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 979820.765013

# use NAAF >50% ALC to get NAAF factor
peaks417['NAAF factor'] = (peaks417['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_417 = peaks417[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_417['A-NAAF50'] = peaksAAPTM_417['A'] * peaks417['NAAF factor']
peaksAAPTM_417['C-NAAF50'] = peaksAAPTM_417['C'] * peaks417['NAAF factor']
peaksAAPTM_417['D-NAAF50'] = peaksAAPTM_417['D'] * peaks417['NAAF factor']
peaksAAPTM_417['E-NAAF50'] = peaksAAPTM_417['E'] * peaks417['NAAF factor']
peaksAAPTM_417['F-NAAF50'] = peaksAAPTM_417['F'] * peaks417['NAAF factor']
peaksAAPTM_417['G-NAAF50'] = peaksAAPTM_417['G'] * peaks417['NAAF factor']
peaksAAPTM_417['H-NAAF50'] = peaksAAPTM_417['H'] * peaks417['NAAF factor']
peaksAAPTM_417['I-NAAF50'] = peaksAAPTM_417['I'] * peaks417['NAAF factor']
peaksAAPTM_417['K-NAAF50'] = peaksAAPTM_417['K'] * peaks417['NAAF factor']
peaksAAPTM_417['L-NAAF50'] = peaksAAPTM_417['L'] * peaks417['NAAF factor']
peaksAAPTM_417['M-NAAF50'] = peaksAAPTM_417['M'] * peaks417['NAAF factor']
peaksAAPTM_417['N-NAAF50'] = peaksAAPTM_417['N'] * peaks417['NAAF factor']
peaksAAPTM_417['P-NAAF50'] = peaksAAPTM_417['P'] * peaks417['NAAF factor']
peaksAAPTM_417['Q-NAAF50'] = peaksAAPTM_417['Q'] * peaks417['NAAF factor']
peaksAAPTM_417['R-NAAF50'] = peaksAAPTM_417['R'] * peaks417['NAAF factor']
peaksAAPTM_417['S-NAAF50'] = peaksAAPTM_417['S'] * peaks417['NAAF factor']
peaksAAPTM_417['T-NAAF50'] = peaksAAPTM_417['T'] * peaks417['NAAF factor']
peaksAAPTM_417['V-NAAF50'] = peaksAAPTM_417['V'] * peaks417['NAAF factor']
peaksAAPTM_417['W-NAAF50'] = peaksAAPTM_417['W'] * peaks417['NAAF factor']
peaksAAPTM_417['Y-NAAF50'] = peaksAAPTM_417['Y'] * peaks417['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_417['ccarb-NAAF50'] = peaksAAPTM_417['c-carb'] * peaksAAPTM_417['NAAF factor']
peaksAAPTM_417['moxid-NAAF50'] = peaksAAPTM_417['m-oxid'] * peaksAAPTM_417['NAAF factor']
peaksAAPTM_417['ndeam-NAAF50'] = peaksAAPTM_417['n-deam'] * peaksAAPTM_417['NAAF factor']
peaksAAPTM_417['qdeam-NAAF50'] = peaksAAPTM_417['q-deam'] * peaksAAPTM_417['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_417.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50_naaf.csv")

peaksAAPTM_417.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,VLEGNEQFLNAAK,0.311674,2,0,0,2,1,1,0,1,...,0.000000,0.000000,0.000000,0.311674,0.0,0.0,0.0,0.000000,0.0,0.0
1,LAADDFR,0.035867,2,0,2,0,1,0,0,0,...,0.035867,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,KMLNAAK,0.050155,2,0,0,0,0,0,0,2,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.050155,0.0,0.0
3,LSSPATLNSR,9.858946,1,0,0,0,0,0,0,0,...,9.858946,29.576838,9.858946,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,LSSPATLNSR,9.858946,1,0,0,0,0,0,0,0,...,9.858946,29.576838,9.858946,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_417['NAAF factor'].sum(),
        'A': peaksAAPTM_417['A-NAAF50'].sum(),
        'C': peaksAAPTM_417['C-NAAF50'].sum(),
        'D': peaksAAPTM_417['D-NAAF50'].sum(),
        'E': peaksAAPTM_417['E-NAAF50'].sum(),
        'F': peaksAAPTM_417['F-NAAF50'].sum(),
        'G': peaksAAPTM_417['G-NAAF50'].sum(),
        'H': peaksAAPTM_417['H-NAAF50'].sum(),
        'I': peaksAAPTM_417['I-NAAF50'].sum(),
        'K': peaksAAPTM_417['K-NAAF50'].sum(),
        'L': peaksAAPTM_417['L-NAAF50'].sum(),
        'M': peaksAAPTM_417['M-NAAF50'].sum(),
        'N': peaksAAPTM_417['N-NAAF50'].sum(),
        'P': peaksAAPTM_417['P-NAAF50'].sum(),
        'Q': peaksAAPTM_417['Q-NAAF50'].sum(),
        'R': peaksAAPTM_417['R-NAAF50'].sum(),
        'S': peaksAAPTM_417['S-NAAF50'].sum(),
        'T': peaksAAPTM_417['T-NAAF50'].sum(),
        'V': peaksAAPTM_417['V-NAAF50'].sum(),
        'W': peaksAAPTM_417['W-NAAF50'].sum(),
        'Y': peaksAAPTM_417['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_417['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_417['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_417['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_417['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 979820.765013

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,275.855601,190.354372,7.189159,5.088807,4.733459,5.330871,18.420685,0.69748,0.0,13.836877,...,1.199389,7.189159,6.263815,3.838516,0.436432,1.0,0.721805,0.070075,0.122416,0.000282


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks417[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks417['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks417[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 259
# nonredundant stripped Peaks peptides >50% ALC 225
average peptide length Peaks peptides >50% ALC 8.787644787644787
# redundant Peaks peptides >50% ALC 259
# nonredundant Peaks peptides 230


,stripped_peptide
0,VLEGNEQFLNAAK
1,LAADDFR
2,KMLNAAK
3,LSSPATLNSR
4,LSSPATLNSR
